## Importaciones necesarias

In [1]:
import pandas as pd
import os 
from typing import Annotated
from openai import AsyncOpenAI
from dotenv import load_dotenv

from semantic_kernel.agents import ChatCompletionAgent, ChatHistoryAgentThread
from semantic_kernel.connectors.ai.open_ai import OpenAIChatCompletion
from semantic_kernel.functions import kernel_function
from semantic_kernel.contents import FunctionCallContent, FunctionResultContent, StreamingTextContent

from IPython.display import display, HTML

In [2]:
load_dotenv()
client = AsyncOpenAI(
    api_key=os.environ.get("GITHUB_TOKEN"), 
    base_url="https://models.inference.ai.azure.com/",
)

# Create an AI Service that will be used by the `ChatCompletionAgent`
chat_completion_service = OpenAIChatCompletion(
    ai_model_id="gpt-4o-mini",
    async_client=client,
      api_key=os.getenv("OPENAI_API_KEY"),
)

In [3]:
api_key = os.getenv("OPENAI_API_KEY")
print("API Key detectada:", "✅" if api_key else "❌ NO detectada")

API Key detectada: ✅


## Clase principal del plugin


*
- Filtra los datos por edad.
- Calcula los promedios de presión sistólica y diastólica.
- Devuelve un string con los valores.

*
- Permite consultar toda la info de un paciente usando el número de fila (índice).
- Verifica que el índice esté dentro del rango.

In [4]:
#from BloodPressurePlugin import BloodPressurePlugin

class BloodPressurePlugin:
    def __init__(self):
        self.df = pd.read_csv("Database/BloodPressuredataset.csv")

    @kernel_function(description="Devuelve el promedio de presión para una edad específica")
    def get_average_pressure_by_age(self, age: Annotated[int, "Edad del paciente"]) -> str:
        filtered_df = self.df[self.df["Edad"] == age]
        if filtered_df.empty:
            return f"No se encontraron datos para edad {age}."
        sys = filtered_df["Presion_Sistolica"].mean()
        dia = filtered_df["Presion_Diastolica"].mean()
        return f"Presión promedio para {age} años: {sys:.2f}/{dia:.2f} mmHg"

    @kernel_function(description="Muestra la información completa de un paciente por índice")
    def get_patient_info_by_index(self, index: Annotated[int, "Índice del paciente"]) -> str:
        if index < 0 or index >= len(self.df):
            return f"Índice fuera de rango. Hay {len(self.df)} pacientes."
        return self.df.iloc[index].to_string()

    @kernel_function(description="Paciente con la presión total más alta")
    def get_highest_pressure_patient(self) -> str:
        self.df["Presion_Total"] = self.df["Presion_Sistolica"] + self.df["Presion_Diastolica"]
        idx = self.df["Presion_Total"].idxmax()
        return self.df.iloc[idx].to_string()

    @kernel_function(description="Promedio de glucosa en todos los pacientes")
    def get_average_glucose(self) -> str:
        avg = self.df["Glucosa"].mean()
        return f"Promedio de glucosa: {avg:.2f} mg/dL"

    @kernel_function(description="Paciente con glucosa más alta")
    def get_max_glucose_patient(self) -> str:
        idx = self.df["Glucosa"].idxmax()
        return self.df.iloc[idx].to_string()

    @kernel_function(description="Devuelve pacientes con nivel de estrés alto")
    def get_high_stress_patients(self) -> str:
        high_stress = self.df[self.df["Nivel_Estres"] > 7]  # supón que >7 es alto
        if high_stress.empty:
            return "No hay pacientes con nivel de estrés alto."
        return high_stress[["Edad", "Nivel_Estres", "Frecuencia_Cardiaca"]].to_string(index=False)

    @kernel_function(description="Resumen general de salud para todos los pacientes")
    def get_general_health_summary(self) -> str:
        summary = self.df.describe().to_string()
        return f"Resumen estadístico general:\n{summary}"

## Funciones decoradas con @kernel_function
Estas funciones son las que el agente puede usar para responder preguntas. 


In [5]:
@kernel_function(description="Devuelve el promedio de presión para una edad específica")
def get_average_pressure_by_age(self, age: Annotated[int, "Edad del paciente"]) -> str:
    filtered_df = self.df[self.df["Edad"] == age]
    if filtered_df.empty:
        return f"No se encontraron datos para edad {age}."
    systolic_avg = filtered_df["Presion_Sistolica"].mean()
    diastolic_avg = filtered_df["Presion_Diastolica"].mean()
    return f"Presión promedio para edad {age}: {systolic_avg:.2f}/{diastolic_avg:.2f} mmHg"

In [6]:
@kernel_function(description="Muestra la información completa de un paciente por índice")
def get_patient_info_by_index(self, index: Annotated[int, "Índice del paciente"]) -> str:
    if index < 0 or index >= len(self.df):
        return f"Índice fuera de rango. Hay {len(self.df)} pacientes."
    patient = self.df.iloc[index]
    return patient.to_string()


In [7]:
@kernel_function(description="Devuelve el paciente con la presión más alta")
def get_highest_pressure_patient(self) -> str:
    self.df["Presion_Total"] = self.df["Presion_Sistolica"] + self.df["Presion_Diastolica"]
    max_index = self.df["Presion_Total"].idxmax()
    return self.df.iloc[max_index].to_string()

* Responder preguntas como “¿Cuál es la presión promedio para una persona de 45 años?”

* Mostrar información específica de un paciente por índice

* Detectar automáticamente al paciente más crítico con mayor presión

### Conectar tu plugin BloodPressurePlugin al agente VitalMind
Asumiendo que ya tenemos el agente VitalMind declarado así:

In [8]:
agent = ChatCompletionAgent(
    service=chat_completion_service,
    plugins=[BloodPressurePlugin()],
    name="VitalMind",
    instructions="Eres un asistente de salud que ayuda a interpretar datos clínicos como presión arterial, glucosa, estrés y más, ofreciendo respuestas útiles, claras y seguras.",
)

In [9]:
# Función para ejecutar preguntas al agente
async def main():
    thread: ChatHistoryAgentThread | None = None

    user_inputs = [
        "Tengo 75 años y me han subido los niveles de glucosa, ¿qué puedo hacer?",
        "Mi padre tiene 82 años y a veces se le olvida tomar sus medicamentos, ¿cómo puedo ayudarlo?",
        "¿Qué debo hacer si tengo presión arterial alta y me siento mareado?",
        "¿Cuál es el nivel promedio de estrés en la base de datos?",
        "¿Qué pacientes tienen colesterol alto según la base?",
    ]

    for user_input in user_inputs:
        print(f"# Usuario: {user_input}\n")
        first_chunk = True
        async for response in agent.invoke_stream(messages=user_input, thread=thread):
            if first_chunk:
                print(f"# VitalMind: ", end="", flush=True)
                first_chunk = False
            print(f"{response}", end="", flush=True)
            thread = response.thread
        print()

    await thread.delete() if thread else None

# Ejecutar el asistente
await main()

# Usuario: Tengo 75 años y me han subido los niveles de glucosa, ¿qué puedo hacer?



ServiceResponseException: ("<class 'semantic_kernel.connectors.ai.open_ai.services.open_ai_chat_completion.OpenAIChatCompletion'> service failed to complete the prompt", AuthenticationError("Error code: 401 - {'error': {'code': 'unauthorized', 'message': 'Bad credentials', 'details': 'Bad credentials'}}"))